In [ ]:
import gradio as gr
import numpy as np
from PIL import Image

def model_a(image):
    # Vit
    return image

def model_b(image):
    # Cnn1
    return image

def model_c(image):
    # Cnn2
    return image

def segment_image(image, model_choice):
    if model_choice == "Model A":
        return model_a(image)
    elif model_choice == "Model B":
        return model_b(image)
    elif model_choice == "Model C":
        return model_c(image)

demo = gr.Interface(
    fn=segment_image,
    inputs=[gr.Image(type="pil"), gr.Dropdown(choices=["Model A", "Model B", "Model C"], label="Select Model")],
    outputs=gr.Image(type="pil")
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
